****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [1]:
# 1.
# 10 countries with the lowest infant mortality rates

#First we import ElementTree since we will be working with xml data, pandas for data frames.
from xml.etree import ElementTree as ET
import pandas as pd

#We then use the .parse() method to read in the data in to the ElementTree
document = ET.parse( './data/mondial_database.xml' )

#first we initialize and empty list, which we will use to select top values.
d = []

#Since every country in the data contains a 'name' value and 'infant_mortality' value we will run a loop to insert all 
#this values in to the 'd' list.  We use the findall() to find all countries, and then find its 'name' and 'infant_mortality'
#values and add them to the list using .append() methond.
for element in document.findall('country'):
    if element.find('infant_mortality') is not None:
    
        d.append([(element.find('name').text),float(element.find('infant_mortality').text)])

#Now that we have all values that we are interested inside the d list, we simply convert it to a Data Frame with columns
# 'Country' and 'Infant_Mortality'.
d1 = pd.DataFrame(d,columns=['Country','Infant_Mortality'])

# We now sort the values according to 'Infant_Mortality' and make 'Country' as index for display purposes.
d1 = d1.set_index('Country').sort_values('Infant_Mortality')

#Now we print the Country name and for the lowest 10 Infant Mortality rates.
d1.head(10)



,Infant_Mortality
Country,
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13


In [2]:
# 2.
# 10 cities with the largest population

# We follow the same steps as previous exercise and create an empty list to store the values needed.
d = []

# Now we find all cities inside and country and store its 'name' and 'population' values in to list 'd'
# Notice that some cities have more than one population value, we are interested in the most recent one(latest entry)
# which will be why [-1] position is used in findall() method.
for element in document.findall('country'):
    for subelement in element.getiterator('city'):
        if subelement.find('population') is not None:
            d.append([(subelement.find('name').text),int(subelement.findall('population')[-1].text)])

# Repeat previous steps to create dataframe           
d1 = pd.DataFrame(d,columns=['City_Name','Population'])

# Sort and display city name and population of 10 largest cities.
d1 = d1.set_index('City_Name').sort_values('Population',ascending=False)
d1.head(10)

,Population
City_Name,
Shanghai,22315474
Istanbul,13710512
Mumbai,12442373
Moskva,11979529
Beijing,11716620
São Paulo,11152344
Tianjin,11090314
Guangzhou,11071424
Delhi,11034555


In [3]:
# 3.
# 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

# We follow the same steps as previous exercise and create an empty list to store the values needed.
d = []

# first we find all countries in the data set.  Once we have country, we then need to find all ethnic groups, since
# most countries have more than one ethnicgroup, we use the findall() method.
for element in document.findall('country'):
    for subelement in element.findall('ethnicgroup'):
        
        #After we have found ethnicgroup in the data. We are intersted in the values of its name, the percentage
        #and total population of country.  With this we can simply calculate the total number of people of that
        #ethnicity in that country. The [-1] location on population is given to state that we need the last value out
        #the 'population' list.
        if element.find('ethnicgroup') is not None:
            ethnicity = subelement.text
            percentage = float(subelement.get('percentage'))
            population = int(element.findall('population')[-1].text)
            total = float(percentage*.01)*(population)
            d.append([ethnicity,total])

#Convert the 'd' with value to a data frame.
d1 = pd.DataFrame(d,columns=['ethnicgroup','totalcount'])

# Now we group by ethnicity and sum all its totalcount to find the top 10 ethnic groups in the data.
d2 = d1.groupby('ethnicgroup')['totalcount'].sum().sort_values(ascending=False).reset_index()
d2.head(10)

,ethnicgroup,totalcount
0,Han Chinese,1.245059e+09
1,Indo-Aryan,8.718156e+08
2,European,4.948722e+08
3,African,3.183251e+08
4,Dravidian,3.027137e+08
5,Mestizo,1.577344e+08
6,Bengali,1.467769e+08
7,Russian,1.318570e+08
8,Japanese,1.265342e+08
9,Malay,1.219936e+08


In [4]:
# 4.
# 4a.name and country longest river.

# create the empty list to add values name, country, length
d = []

# Like previous examples, we find all rivers in the xml data
for element in document.findall('river'):
        
        #for each river we find, we make sure that it hast a 'length' value.  We then store name, length, country in the the
        # 'd' list.
        if element.find('length') is not None:
            name = element.find('name').text
            length = float(element.find('length').text)
            country = element.get('country')
            d.append([name,country,length])

# Now that we have all the values of interest, we set them to a data frame
d1 = pd.DataFrame(d,columns=['River_Name','Country', 'Length'])

# we now sort the data frame according to length value in descending order to get the higheset at top.  
# And finally print out the top value
d2 = d1.sort_values('Length',ascending=False)
d2.head(1)

,River_Name,Country,Length
174,Amazonas,CO BR PE,6448.0


In [5]:
# 4.
# 4b.name and country largest lake.

# empty list to add values of name, country and area of lakes.
d = []

# find all lakes in the data and add values of interest(name, country, area) to 'd' list
for element in document.findall('lake'):

        if element.find('area') is not None:
            name = element.find('name').text
            area = float(element.find('area').text)
            country = element.get('country')
            d.append([name,country,area])
#convert to dataframe
d1 = pd.DataFrame(d,columns=['Lake_Name','Country', 'Area'])

# sort the dataframe by its 'Area' values in descending order and print out its top row.
d2 = d1.sort_values('Area',ascending=False)
d2.head(1)

,Lake_Name,Country,Area
54,Caspian Sea,R AZ KAZ IR TM,386400.0


In [6]:
# 4.
# 4c.name and country of airport at highest elevation

# Emptly list to add values of interest(name, country, elevation)
d = []

# Find all airports in the data and collect information on its name, country and elevation.
for element in document.findall('airport'):

        if element.find('elevation').text is not None:
            name = element.find('name').text
            elevation = float(element.find('elevation').text)
            country = element.get('country')
            d.append([name,country,elevation])

# convert to dataframe with values 'Lake_Name', 'Country', 'Elevation'.
d1 = pd.DataFrame(d,columns=['Lake_Name','Country', 'Elevation'])

# sort by elevation value in descending order and print out the highest elevation airport in the data.
d2 = d1.sort_values('Elevation',ascending=False)
d2.head(1)

,Lake_Name,Country,Elevation
80,El Alto Intl,BOL,4063.0
